### Part A

Naive Bayes classifier is an algorithm that uses Bayes' theorem to classify objects. It's called Naive because it makes an assumption that each feature is independent of the other.

In [3]:
#Importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
#Loading the dataset into a pandas data frame
df = pd.read_csv('./adult.csv')
# df.head()
df.isnull().sum().sum()

#Dropping education as educational num cover it
df.drop(["education"], axis = 1,  inplace=True)

#Converting the last column to 0s and 1s
df['income'].replace(['>50K', '<=50K'], [1, 0], inplace=True)

# Replacing ? with the most frequent value in each column
df['age'].replace( "?", df['age'].mode()[0], inplace=True)
df['workclass'].replace("?", df['workclass'].mode()[0], inplace=True)
df['fnlwgt'].replace("?", df['fnlwgt'].mode()[0], inplace=True)
df['educational-num'].replace("?",
                              df['educational-num'].mode()[0], inplace=True)
df['occupation'].replace("?", df['occupation'].mode()[0], inplace=True)
df['race'].replace("?", df['race'].mode()[0], inplace=True)
df['gender'].replace("?", df['gender'].mode()[0], inplace=True)
df['capital-gain'].replace("?", df['capital-gain'].mode()[0], inplace=True)
df['capital-loss'].replace("?", df['capital-loss'].mode()[0], inplace=True)
df['hours-per-week'].replace("?", df['hours-per-week'].mode()[0], inplace=True)
df['native-country'].replace("?", df['native-country'].mode()[0], inplace=True)


#Converting categorical variables to numberic values
df['workclass'].replace(df.workclass.unique(), [
                        i for i in range(len(df['workclass'].unique()))], inplace=True)
df['marital-status'].replace(df['marital-status'].unique(), [
    i for i in range(len((df['marital-status'].unique())))], inplace=True)
df['relationship'].replace(df['relationship'].unique(), [
    i for i in range(len(df['relationship'].unique()))], inplace=True)
df['gender'].replace(df['gender'].unique(), [
    i for i in range(len(df['gender'].unique()))], inplace=True)
df['native-country'].replace(df['native-country'].unique(), [
    i for i in range(len(df['native-country'].unique()))], inplace=True)
df['occupation'].replace(df['occupation'].unique(), [
    i for i in range(len(df['occupation'].unique()))], inplace=True)
df['race'].replace(df['race'].unique(), [
    i for i in range(len(df['race'].unique()))], inplace=True)

df.head()





,age,workclass,fnlwgt,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,0,226802,7,0,0,0,0,0,0,0,40,0,0
1,38,0,89814,9,1,1,1,1,0,0,0,50,0,0
2,28,1,336951,12,1,2,1,1,0,0,0,40,0,1
3,44,0,160323,10,1,0,1,0,0,7688,0,40,0,1
4,18,0,103497,10,0,3,0,1,1,0,0,30,0,0


In [58]:
df["fnlwgt"].value_counts()[2]

KeyError: 2

In [61]:
from sklearn.metrics import f1_score


class NaiveBayes():
    def __init__(self, dataset):
        # Initializing the dataset
        self.dataset = dataset

        # Splitting training and testing datasets
        self.training_dataset = self.dataset.iloc[:int(
            0.80*self.dataset.shape[0]), :]
        self.testing_dataset = self.dataset.iloc[int(
            0.20*self.dataset.shape[0]):, :]

        self.X_train = self.training_dataset.iloc[:, :-1]
        self.Y_train = self.training_dataset.iloc[:, -1]
        self.X_test = self.testing_dataset.iloc[:, :-1]
        self.Y_test = self.testing_dataset.iloc[:, -1]

    def calculate_prior(self):
        income_classes = sorted(list(df['income'].unique()))
        self.prior_probability = []
        for i in income_classes:
            self.prior_probability.append(len(df[df['income'] == i])/len(df))

    def calculate_likelihood(self, feature_name, feature_val, output_needed):
        # feature_name = column name
        # feature_val = value needed
        # output_needed = 1 or 0
        df = self.dataset[self.dataset["income"] == output_needed]
        # print(df[feature_name].value_counts().113364)
        if feature_val in df[feature_name].values:
            count = df[feature_name].value_counts()[feature_val]
        else:
            count = 0
            
        p_x_given_y = count/df.shape[0]
        return p_x_given_y

    def naive_bayes(self):
        features = list(self.dataset.columns)[:-1]
        self.calculate_prior()

        y_pred = []

        # We are storing labels as output -> meaning 0 means probability in 0 world and 1 means probability in 1 world
        labels = [0,1]
        # We initialize it to 1
        likelihood = [1 for i in range(len(labels))]
        # For every row
        for row in self.X_train.values:
                # we iterate through 0 and 1 worlds
                for j in range(len(labels)):
                    # then iterate through all the features and multiply with the likelihood
                    for i in range(len(features)):
                        # print(features[i], row[i])
                        likelihood[j] *= self.calculate_likelihood(features[i], row[i], labels[j])

        # Calculating posterior probability (only numerator)
        self.posterior_probability = [1 for i in range(len(labels))]
        for j in range(len(labels)):
            self.posterior_probability = likelihood[j] * \
                self.prior_probability[j]
    
        y_pred.append(np.argmax(self.posterior_probability))

        return np.array(y_pred)


In [62]:
nb = NaiveBayes(df)
# y_test = nb.Y_test
y_pred = nb.naive_bayes()
# print(f1_score(nb.Y_test, y_pred))



KeyboardInterrupt: 